In [1]:
import pandas as pd
import numpy as np
np.random.seed(32)
import sqlite3
import time
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### for fast text

In [2]:
from array import array

In [4]:
db = sqlite3.connect(r"I:\DataSets\NLP\Embeddings_DB\wiki.en.db")

In [8]:
# for glove
db = sqlite3.connect(r"I:\DataSets\NLP\Embeddings_DB\glove.840B.300d.db")

In [9]:
c = db.cursor()

In [5]:
def lookup(word):
    q = c.execute('select emb from embeddings where word = :word', {'word': word}).fetchone()
    return array('f', q[0]).tolist() if q else None

In [12]:
embed_size = 300
max_features = 200000
max_len = 300

In [3]:
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [6]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
train["comment_text"].fillna("fillna")
test["comment_text"].fillna("fillna")
X_train, X_valid, Y_train, Y_valid = train_test_split(train, y, test_size = 0.1)

In [7]:
raw_text_train = X_train["comment_text"].astype(str)
raw_text_valid = X_valid["comment_text"].astype(str)
raw_text_test = test["comment_text"].astype(str)

tk = Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid)
test["comment_seq"] = tk.texts_to_sequences(raw_text_test)

X_train = pad_sequences(X_train.comment_seq, maxlen = max_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen = max_len)
test = pad_sequences(test.comment_seq, maxlen = max_len)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
# for fast text max_len = 200, max_features = 100000
word_index = tk.word_index
print(len(word_index))
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = lookup(word)
    if embedding_vector is not None: embedding_matrix[i] = np.array(embedding_vector)

172408


In [21]:
embedding_matrix.shape

(100000, 300)

In [10]:
# for glove max_len = 300, max_features = 200000
word_index = tk.word_index
print(len(word_index))
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
co = 0
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = lookup(word)
    if embedding_vector is not None:
        embedding_matrix[i] = np.array(embedding_vector)
        co = co + 1

172408


IndexError: index 172408 is out of bounds for axis 0 with size 172408

In [22]:
import pickle

In [23]:
with open('ft.wiki.en.pickle', 'wb') as handle:
    pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
del embedding_matrix

In [27]:
c.close()

In [28]:
db.close()

In [29]:
db1 = sqlite3.connect(r"I:\DataSets\NLP\Embeddings_DB\wiki.en.db\glove.840B.300d.db")

OperationalError: unable to open database file

In [30]:
with open('X_train.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('X_valid.pickle', 'wb') as handle:
    pickle.dump(X_valid, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open('test.pickle', 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
with open('Y_train.pickle', 'wb') as handle:
    pickle.dump(Y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
with open('Y_valid.pickle', 'wb') as handle:
    pickle.dump(Y_valid, handle, protocol=pickle.HIGHEST_PROTOCOL)

### for glove